## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Havelock,US,34.8791,-76.9013,88.05,80,100,5.01,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.54,73,12,4.25,few clouds
2,2,Bethel,US,41.3712,-73.4140,73.85,89,91,7.85,overcast clouds
3,3,Hami,CN,42.8000,93.4500,79.72,22,0,4.92,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,80.56,81,50,9.84,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum temperature criteria?"))
max_temp = float(input("What is your maximum temperature criteria?"))

What is your minimum temperature criteria?70
What is your maximum temperature criteria?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Havelock,US,34.8791,-76.9013,88.05,80,100,5.01,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.54,73,12,4.25,few clouds
2,2,Bethel,US,41.3712,-73.4140,73.85,89,91,7.85,overcast clouds
3,3,Hami,CN,42.8000,93.4500,79.72,22,0,4.92,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,80.56,81,50,9.84,scattered clouds
6,6,Vila Franca Do Campo,PT,37.7167,-25.4333,70.21,80,54,8.70,broken clouds
8,8,Salalah,OM,17.0151,54.0924,78.87,88,100,5.12,overcast clouds
10,10,Acarau,BR,-2.8856,-40.1200,79.74,74,35,15.19,scattered clouds
11,11,Rikitea,PF,-23.1203,-134.9692,74.10,73,6,13.71,clear sky
12,12,Albany,US,42.6001,-73.9662,79.18,83,98,11.56,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                340
City                   340
Country                340
Lat                    340
Lng                    340
Max Temp               340
Humidity               340
Cloudiness             340
Wind Speed             340
Current Description    340
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Havelock,US,34.8791,-76.9013,88.05,80,100,5.01,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.54,73,12,4.25,few clouds
2,2,Bethel,US,41.3712,-73.4140,73.85,89,91,7.85,overcast clouds
3,3,Hami,CN,42.8000,93.4500,79.72,22,0,4.92,clear sky
4,4,Puerto Ayora,EC,-0.7393,-90.3518,80.56,81,50,9.84,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Havelock,US,88.05,overcast clouds,34.8791,-76.9013,
1,Hithadhoo,MV,81.54,few clouds,-0.6000,73.0833,
2,Bethel,US,73.85,overcast clouds,41.3712,-73.4140,
3,Hami,CN,79.72,clear sky,42.8000,93.4500,
4,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
6,Vila Franca Do Campo,PT,70.21,broken clouds,37.7167,-25.4333,
8,Salalah,OM,78.87,overcast clouds,17.0151,54.0924,
10,Acarau,BR,79.74,scattered clouds,-2.8856,-40.1200,
11,Rikitea,PF,74.10,clear sky,-23.1203,-134.9692,
12,Albany,US,79.18,overcast clouds,42.6001,-73.9662,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Havelock,US,88.05,overcast clouds,34.8791,-76.9013,Sherwood Motel
1,Hithadhoo,MV,81.54,few clouds,-0.6000,73.0833,Scoop Guest House
2,Bethel,US,73.85,overcast clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
3,Hami,CN,79.72,clear sky,42.8000,93.4500,Laoyutang Resort
4,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [10]:
# 8a. Create the output File (CSV)

output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))